# Financial Reform Indicators

A New Database of Financial Reforms: https://www.imf.org/external/pubs/cat/longres.aspx?sk=22485.0

In [2]:
import requests
import zipfile

import numpy as np
import pandas as pd

pd.set_option('float_format', '{:6.2f}'.format)

Download data and save it to disk. Also consider using ``urllib`` and ``urllib2``.

In [3]:
url = 'https://www.imf.org/external/pubs/ft/wp/2008/Data/wp08266.zip'
path = '../data/IMF_Fin_Reform/fin_reform.zip'

response = requests.get(url)
with open(path, "wb") as file:
    file.write(response.content)

Display contents of the zip file.

In [4]:
zf = zipfile.ZipFile(path)
files = zf.namelist()

print(files)

['Financial Reform Dataset Dec 2008.dta', 'Financial Reform Dataset Dec 2008.xls']


Import `Readme` list from Excel file which contains variable names and long labels.

In [5]:
readme = pd.read_excel(zf.open(files[1]), 'Readme', skiprows=6, index_col=0).dropna()

print(readme)

                                                       Description
Variable                                                          
country                                               Country name
ifs                                       IFS 3-digit country code
wdi                                      WDI 3-letter country code
year                                                          Year
directedcredit                Directed credit/reserve requirements
creditceilings                           Aggregate Credit Ceilings
creditcontrols   Credit Controls, defined as 0.75*DIRECTEDCREDI...
intratecontrols                             Interest rate controls
entrybarriers              Entry barriers/pro-competition measures
bankingsuperv                                  Banking Supervision
privatization                                        Privatization
intlcapital                            International capital flows
securitymarkets                                   Security Mar

Load data from `Stata` `*.dta` file. Display data types.

In [6]:
raw = pd.read_stata(zf.open(files[0]))

print(raw.dtypes)

country             object
ifs                  int16
wdi                 object
year                 int16
directedcredit        int8
creditceilings     float64
creditcontrols     float32
intratecontrols       int8
entrybarriers         int8
bankingsuperv         int8
privatization         int8
intlcapital           int8
securitymarkets       int8
finreform          float32
finreform_n        float32
large_reversal     float64
reversal           float64
reform             float64
large_reform       float64
status_quo         float64
Advanced              int8
Emerging_Asia         int8
Latin_America         int8
SSA                   int8
Transition            int8
MENA                  int8
dtype: object


Combine region dummies into one categorical index.

More details on categorical data can be found here: http://pandas.pydata.org/pandas-docs/stable/categorical.html

In [128]:
data = raw.copy()
categories = list(readme.index[-6:])
data['Region'] = np.nan

for category in categories:
    data[category] = pd.Categorical.from_codes(data[category], categories=['non-' + category, category])
    condition = data[category] == category
    data.ix[condition, 'Region'] = data.ix[condition, category]

data['Region'] = data['Region'].astype('category')

print(data['Region'].describe())

count         2671
unique           6
top       Advanced
freq           726
Name: Region, dtype: object


Index data by regions, countries, and years. Sort index. Show overall descriptive statistics.

In [130]:
categories.append('Region')
categories.extend(['country', 'year'])
print(categories)

data = data.set_index(categories)[['finreform_n']].sort_index() * 100

print(data.describe())

['Advanced', 'Emerging_Asia', 'Latin_America', 'SSA', 'Transition', 'MENA', 'Region', 'country', 'year']
       finreform_n
count      2671.00
mean         49.15
std          30.15
min           0.00
25%          22.62
50%          50.00
75%          73.81
max         100.00


How many countries do we have?

In [131]:
countries = data.index.get_level_values('country').unique()

print(countries)
print(len(countries))

['Australia' 'Austria' 'Belgium' 'Britain' 'Canada' 'Denmark' 'Finland'
 'France' 'Germany' 'Greece' 'Ireland' 'Israel' 'Italy' 'Japan'
 'Netherlands' 'New Zealand' 'Norway' 'Portugal' 'Spain' 'Sweden'
 'Switzerland' 'United States' 'Bangladesh' 'China' 'Hong Kong' 'India'
 'Indonesia' 'Korea' 'Malaysia' 'Nepal' 'Philippines' 'Singapore'
 'Sri Lanka' 'Taiwan' 'Thailand' 'Argentina' 'Bolivia' 'Brazil' 'Chile'
 'Colombia' 'Costa Rica' 'Dominican Rep' 'Ecuador' 'El Salvador'
 'Guatemala' 'Jamaica' 'Mexico' 'Nicaragua' 'Paraguay' 'Peru' 'Uruguay'
 'Venezuela' 'Burkina-Faso' 'Cameroon' 'Cote d Ivoire' 'Ethiopia' 'Ghana'
 'Kenya' 'Madagascar' 'Mozambique' 'Nigeria' 'Senegal' 'South Africa'
 'Tanzania' 'Uganda' 'Zimbabwe' 'Albania' 'Azerbaijan' 'Belarus' 'Bulgaria'
 'Czech Rep' 'Estonia' 'Georgia' 'Hungary' 'Kazakhstan' 'Kyrgyz Rep'
 'Latvia' 'Lithuania' 'Poland' 'Romania' 'Russia' 'Ukraine' 'Uzbekistan'
 'Vietnam' 'Algeria' 'Egypt' 'Jordan' 'Morocco' 'Pakistan' 'Tunisia'
 'Turkey']
91


How many year observations do we have for each country?

In [132]:
counts = data.reset_index('year')['year'].groupby(level='country').count()

print(counts.describe())

count    91.00
mean     29.35
std       7.22
min      13.00
25%      33.00
50%      33.00
75%      33.00
max      33.00
Name: year, dtype: float64


Which countries have the most missing values?

In [133]:
print(counts.order().head())

country
Russia        13
Latvia        14
Lithuania     14
Belarus       14
Kazakhstan    15
Name: year, dtype: int64


Compute the change in financial reform index year-to-year for each country.

In [134]:
data['finreform change'] = data.groupby(level=categories[:-1]).diff()

print(data.reset_index(categories[:-3], drop=True).head())

                         finreform_n  finreform change
Region   country   year                               
Advanced Australia 1973         9.52               nan
                   1974         9.52              0.00
                   1975         9.52              0.00
                   1976         9.52              0.00
                   1977         9.52              0.00


Compute average change in financial reform indicator.

In [135]:
table = data.groupby(level=categories[:-1])[['finreform change']].mean()
table.sort_index(by='finreform change', inplace=True)

print(table.reset_index(categories[:-3], drop=True).head())

                           finreform change
Region        country                      
SSA           Ethiopia                 0.00
Advanced      Switzerland              0.45
              Germany                  0.89
Emerging_Asia Nepal                    1.04
              Hong Kong                1.04


Print descriptive statistics for the change in financial reform indicator.

In [136]:
print(table['finreform change'].describe())

count    91.00
mean      2.22
std       1.09
min       0.00
25%       1.58
50%       1.93
75%       2.48
max       5.68
Name: finreform change, dtype: float64


Divide countries into three equal groups by the speed of financial reforms.

In [137]:
table['change group'] = pd.cut(table['finreform change'], 3, labels=['slow', 'medium', 'fast'])

print(table.reset_index(categories[:-3], drop=True).head())

                           finreform change change group
Region        country                                   
SSA           Ethiopia                 0.00         slow
Advanced      Switzerland              0.45         slow
              Germany                  0.89         slow
Emerging_Asia Nepal                    1.04         slow
              Hong Kong                1.04         slow


Compare the number of countries by speed of financial reforms and by (non)advanced.

In [138]:
table2 = table.set_index('change group', append=True)
table2 = table2.groupby(level=['Advanced', 'change group'])['finreform change'].count()
table2 = table2.unstack('Advanced')

print(table2)

Advanced      Advanced  non-Advanced
change group                        
slow             11.00         30.00
medium           11.00         28.00
fast               nan         11.00


Same for (non)transiton countries.

In [139]:
table2 = table.set_index('change group', append=True)
table2 = table2.groupby(level=['Transition', 'change group'])['finreform change'].count()
table2 = table2.unstack('Transition')

print(table2)

Transition    Transition  non-Transition
change group                            
slow                 nan           41.00
medium              7.00           32.00
fast               11.00             nan


Print descriptive statistics of financial reform changes by region.

In [140]:
table2 = table.groupby(level='Region')['finreform change'].describe().unstack('Region')

print(table2)

Region  Advanced  Emerging_Asia  Latin_America   MENA    SSA  Transition
count      22.00          13.00          17.00   7.00  14.00       18.00
mean        1.82           1.70           1.99   1.75   1.62        3.96
std         0.59           0.41           0.44   0.37   0.59        1.08
min         0.45           1.04           1.38   1.30   0.00        2.20
25%         1.53           1.49           1.64   1.45   1.47        3.02
50%         1.92           1.79           1.93   1.79   1.67        3.98
75%         2.21           1.93           2.23   2.01   1.93        4.84
max         2.83           2.38           2.83   2.23   2.53        5.68


Which country has the highest pace of implementing finacial reforms among transition countries?

In [141]:
table3 = table.xs('Transition', level='Transition')
table3 = table3.sort_index(by='finreform change', ascending=False)

print(table3.reset_index(table3.index.names[:-1], drop=True)['finreform change'].head())

country
Lithuania     5.68
Albania       5.44
Romania       5.32
Latvia        5.31
Czech Rep     4.92
Name: finreform change, dtype: float32
